# Projet Du S1
### Objectif final : Prédiction de l'occupation des lits en fonction de la période ou du service 
La base de ce DataSet est de pouvoir prédire la satisfaction du client en fonction des 6 autres variables 


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

### Import des librairies

In [ ]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

C:\Users\cleme\AppData\Local\Temp\ipykernel_22376\3660000183.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### data set

In [7]:
patients =pd.read_csv("C:/Users/cleme/Documents/Projet_S1_HBM/patients.csv")
patients.head()

,patient_id,name,age,arrival_date,departure_date,service,satisfaction
0,PAT-09484753,Richard Rodriguez,24,2025-03-16,2025-03-22,surgery,61
1,PAT-f0644084,Shannon Walker,6,2025-12-13,2025-12-14,surgery,83
2,PAT-ac6162e4,Julia Torres,24,2025-06-29,2025-07-05,general_medicine,83
3,PAT-3dda2bb5,Crystal Johnson,32,2025-10-12,2025-10-23,emergency,81
4,PAT-08591375,Garrett Lin,25,2025-02-18,2025-02-25,ICU,76


In [ ]:
patients.shape
#1000 patients 7 colonnes

(1000, 7)

In [ ]:
patients.columns

Index(['patient_id', 'name', 'age', 'arrival_date', 'departure_date',
       'service', 'satisfaction'],
      dtype='object')

In [ ]:
features = ['patient_id','name','age','arrival_date','departure_date','service']